In [4]:
import pandas as pd
def find_duplicate_recommendations(df):
    """
    Legge un file CSV di raccomandazioni e trova gruppi di utenti
    che hanno raccomandato esattamente lo stesso set di giochi.

    Args:
        file_path (str or file-like object): Percorso del file CSV o oggetto file.

    Returns:
        pandas.Series: Una serie contenente i gruppi di utenti duplicati,
                       o None se si verifica un errore.
    """

    print("Raggruppamento delle raccomandazioni per utente...")
    # 1. Raggruppa per 'user_id' e crea una lista di 'app_id' per ogni utente.
    user_to_apps_list = df.groupby('user_id')['app_id'].apply(list)

    print("Creazione della 'firma' per ogni profilo utente...")
    # 2. Converte ogni lista di app_id in un 'frozenset'.
    #    Il frozenset ignora l'ordine e i duplicati, creando una "firma" unica
    #    del profilo di raccomandazioni di un utente.
    user_profile_signatures = user_to_apps_list.apply(frozenset)

    print("Identificazione dei gruppi di utenti duplicati...")
    # 3. Raggruppa gli utenti in base alla loro firma.
    #    Il risultato è una Serie dove l'indice è la firma (il set di giochi)
    #    e i valori sono la lista di user_id che condividono quella firma.
    signature_to_users_list = user_profile_signatures.groupby(user_profile_signatures).apply(list)
    
    # 4. Filtra per tenere solo i gruppi con più di un utente (i duplicati).
    duplicate_groups = signature_to_users_list[signature_to_users_list.str.len() > 1]
    
    return duplicate_groups

In [2]:
games = pd.read_csv('games.csv')
recommendations = pd.read_csv('recommendations.csv')
users = pd.read_csv('users.csv')

print(f'Games:{len(games)}')
print(f'Recommendations:{len(recommendations)}')
print(f'Users:{len(users)}')

Games:40499
Recommendations:10338746
Users:840248


In [5]:
groups = find_duplicate_recommendations(recommendations)

if groups is not None:
    print("\n--- RISULTATO DELL'ANALISI ---")
    if groups.empty:
        print("✅ Non sono stati trovati gruppi di utenti con le stesse identiche raccomandazioni.")
    else:
        print(f"🔥 Trovati {len(groups)} gruppi di utenti con raccomandazioni identiche:\n")
        
        # Converte la Serie in un dizionario per una stampa più chiara
        groups_dict = groups.to_dict()
        i = 0
        for app_set, user_list in groups_dict.items():
            i += 1
            # Convertiamo il frozenset in una lista di stringhe per la stampa
            app_ids_str = ', '.join(map(str, list(app_set)))
            print(f"Gruppo {i} (Contiene {len(user_list)} utenti):")
            print(f"  - Set di giochi condiviso (app_id): {app_ids_str}")
            print(f"  - User IDs: {', '.join(map(str, user_list))}\n")


Raggruppamento delle raccomandazioni per utente...
Creazione della 'firma' per ogni profilo utente...
Identificazione dei gruppi di utenti duplicati...

--- RISULTATO DELL'ANALISI ---
🔥 Trovati 99 gruppi di utenti con raccomandazioni identiche:

Gruppo 1 (Contiene 2 utenti):
  - Set di giochi condiviso (app_id): 288160, 1049410, 1361320, 425580, 456750
  - User IDs: frozenset({288160, 1049410, 1361320, 425580, 456750}), frozenset({288160, 1049410, 1361320, 425580, 456750})

Gruppo 2 (Contiene 2 utenti):
  - Set di giochi condiviso (app_id): 47780, 1238060, 254700, 268050, 17470
  - User IDs: frozenset({47780, 1238060, 254700, 268050, 17470}), frozenset({47780, 1238060, 254700, 268050, 17470})

Gruppo 3 (Contiene 2 utenti):
  - Set di giochi condiviso (app_id): 1052100, 640860, 1391690, 623340, 1180110, 1300700
  - User IDs: frozenset({1052100, 640860, 1391690, 623340, 1180110, 1300700}), frozenset({1052100, 1391690, 623340, 1180110, 1300700, 640860})

Gruppo 4 (Contiene 2 utenti):
  - 